In [1]:
#Auxiliares
import os
import pickle
from time import time
from datetime import datetime
from json import dumps

#Dados
import pandas as pd

#preprocessing and transformation
from sklearn.preprocessing import MaxAbsScaler

#Machine learning
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

#Metrics
from sklearn.metrics import classification_report

seed = 42

In [4]:
#Variables
base_path = 'C:/Users/arthu/Desktop/22032020 - Experimentos/05. Organizado/03. Datasets/Clareza-Balanced-Multiclass-COH-METRIX'
save_path = 'output'

x_train_file = 'X_train.csv'
y_train_file = 'y_train.csv'
x_test_file = 'X_test.csv'
y_test_file = 'y_test.csv'

data='resp-text'
label='Clareza'

#Load data
X_train = pd.read_csv(os.path.join(base_path, x_train_file), sep=';', encoding='utf-8')
y_train = pd.read_csv(os.path.join(base_path, y_train_file), sep=';', encoding='utf-8')
X_test = pd.read_csv(os.path.join(base_path, x_test_file), sep=';', encoding='utf-8')
y_test = pd.read_csv(os.path.join(base_path, y_test_file), sep=';', encoding='utf-8')

#Checking on data
print(X_train.columns)
print(X_train.shape)
print(y_train[label].value_counts())
print(y_test[label].value_counts())

Index(['pid', 'req-text', 'resp-text', '1funct-request', '2pronoun-request',
       '3ppron-request', '4i-request', '5we-request', '6you-request',
       '7shehe-request',
       ...
       'sentence_length_min', 'sentence_length_standard_deviation',
       'short_sentence_ratio', 'std_noun_phrase', 'verb_diversity',
       'verbs_max', 'verbs_min', 'verbs_standard_deviation',
       'long_sentence_ratio', 'tempo_resposta'],
      dtype='object', length=310)
(4832, 310)
0    1628
1    1623
2    1581
Name: Clareza, dtype: int64
2    720
1    678
0    673
Name: Clareza, dtype: int64


In [5]:
# Define pipeline and GridSearch CV

params = {
    'vect_analyzer': 'word',
    'vect_max_features': None,
    'vect_min_df': 1,
    'vect_max_df': 0.75,
    'vect_ngram_range': (1,2),
    'tfidf_use_idf': True,
    'tf_idf_norm': 'l2',
    'clf_solver':'lbfgs',
    'clf_max_iter': 10000,
    #'clf_C': 100,
    'gs_cv': 10,
    'gs_scoring': 'f1_macro'   
}

pipelineWord = Pipeline([
    ('vect',   CountVectorizer(analyzer=params.get('vect_analyzer'),
                               max_features=params.get('vect_max_features'),
                               min_df=params.get('vect_min_df'),
                               max_df=params.get('vect_max_df'),
                               ngram_range=params.get('vect_ngram_range'))),
    
    ('tfidf', TfidfTransformer(use_idf=params.get('tfidf_use_idf'),
                               norm=params.get('tf_idf_norm'))),
    
    ('scaler', MaxAbsScaler()),
    
    ('clf', LogisticRegression(random_state=seed,
                               n_jobs=6,
                               C=params.get('clf_C'),
                               solver=params.get('clf_solver'),
                               max_iter=params.get('clf_max_iter')))
])

gs_parameters = {
    #'vect__min_df': (1, 0.1, 0.25),
    #'vect__max_df': (1.0, 0.75),
    'clf__C': (0.1, 1, 10, 100)
}

# Define grid search
grid_search_word = GridSearchCV(pipelineWord,
                               gs_parameters,
                               cv=params.get('gs_cv'),
                               scoring=params.get('gs_scoring'),
                               n_jobs=6,
                               verbose=10
                               )

In [7]:
X = X_train[data]
Y = y_train[label]
x = X_test[data]
y = y_test[label]

In [8]:
print("Executando Gridsearch para Respostas - Multiclass WORD - Classe Clareza - Coh Metrix")

now = str(datetime.now()).split('.')[0].replace('-', '_').replace(' ', '_').replace(':', '_')

t0 = time()
grid_search_word.fit(X, Y)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search_word.best_score_)
print("Best parameters set:")
best_parameters = grid_search_word.best_estimator_.get_params()
for param_name in sorted(gs_parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Executando Gridsearch para Respostas - Multiclass WORD - Classe Clareza - Coh Metrix
Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done   1 tasks      | elapsed:   32.3s
[Parallel(n_jobs=6)]: Done   6 tasks      | elapsed:   33.3s
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done  20 tasks      | elapsed:  3.2min
[Parallel(n_jobs=6)]: Done  29 tasks      | elapsed:  7.6min
[Parallel(n_jobs=6)]: Done  34 out of  40 | elapsed: 14.1min remaining:  2.5min
[Parallel(n_jobs=6)]: Done  40 out of  40 | elapsed: 18.3min finished


done in 1112.040s
Best score: 0.410
Best parameters set:
	clf__C: 0.1


In [9]:
# Saving Model
f_save = now + '_Clareza_Resposta_Multiclass_Balanced_word_coh_metrix.sav'
pickle.dump(grid_search_word, open(os.path.join(save_path, f_save),'wb'))

# Saving Parameters
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\n'+f_save + '\n\n')
    f.write('Parameters:\n')
    f.write(dumps(params) + '\n')
    f.write('\nGridSearch Best Parameters:\n')
    for param_name in sorted(gs_parameters.keys()):        
        f.write("%s: %r" % (param_name, best_parameters[param_name]) + '\n')    

# Validation

In [10]:
# Predictions
y_pred_train = grid_search_word.predict(X)
y_pred_test = grid_search_word.predict(x)

In [11]:
print('#'*50)
print('Report for TRAIN')
print('#'*50)
print(classification_report(Y, y_pred_train))

print('#'*50)
print('Report for TEST')
print('#'*50)
print(classification_report(y, y_pred_test))

##################################################
Report for TRAIN
##################################################
              precision    recall  f1-score   support

           0       0.91      0.96      0.93      1628
           1       0.97      0.92      0.94      1623
           2       0.96      0.94      0.95      1581

    accuracy                           0.94      4832
   macro avg       0.94      0.94      0.94      4832
weighted avg       0.94      0.94      0.94      4832

##################################################
Report for TEST
##################################################
              precision    recall  f1-score   support

           0       0.42      0.45      0.43       673
           1       0.37      0.38      0.38       678
           2       0.42      0.38      0.40       720

    accuracy                           0.40      2071
   macro avg       0.40      0.40      0.40      2071
weighted avg       0.40      0.40      0.40      2071



In [9]:
with open(os.path.join(save_path, 'params.txt'),'a') as f:
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TRAIN')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(Y, y_pred_train))
    
    f.write('\n\n' + ('#'*60))
    f.write('\nReport for TEST')
    f.write('\n' + ('#'*60))
    f.write('\n' + classification_report(y, y_pred_test))